<a href="https://colab.research.google.com/github/victorknox/RedditAnalysis/blob/main/Reddit_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install praw
!pip install asyncpraw

     |████████████████████████████████| 167 kB 4.0 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 172 kB 4.2 MB/s 
     |████████████████████████████████| 1.3 MB 42.1 MB/s 
     |████████████████████████████████| 294 kB 51.0 MB/s 
     |████████████████████████████████| 142 kB 51.8 MB/s 


Creating a read only reddit instance


In [ ]:
import praw
reddit = praw.Reddit(client_id='zlMUUUA0MNikfTR-guoZPg', client_secret='vcgEzBynk1xZHyxaHdxDJ5v67M5_DQ', user_agent='scraper')


## Scraping comments of a post

In [ ]:
submission = reddit.submission(url="https://old.reddit.com/r/movies/comments/osi6v3/lucasfilm_hires_the_youtube_deepfaker_who_put_its/")
# # or 
# submission = reddit.submission(id="a3p0uq")

- Scraping top level comments of a post

In [ ]:
submission.comments.replace_more(limit=0)
for top_level_comment in submission.comments:
    print(top_level_comment.body)

- Scraping all comments of a post

In [ ]:
comments = []

submission.comments.replace_more(limit=0)
for comment in submission.comments.list():
  comment_details = {
    'title':submission.title,
    'body':comment.body,
    'votes':comment.score,
    'subreddit name':submission.subreddit.display_name,
    'time':comment.created_utc,
    'author': comment.author,
    }
  comments.append(comment_details)

print(comments[0])

{'title': 'Lucasfilm hires the YouTube deepfaker who put its Luke, Leia and Tarkin cameos to shame', 'body': "i thought this guy was just a hobbyist youtuber but he's a professional. he's writing papers on all his videos. that's why they hired him. he's not some amateur who is really good at deepfake vids. he's a researcher doing cutting edge work on it.", 'votes': 16651, 'subreddit name': 'movies', 'time': 1627380567.0, 'author': None}


## Scraping comments of urls in url_list



In [ ]:
url_list = ["https://www.reddit.com/r/DeepFakesSFW/comments/mljhi0/everyone_hates_this_on_rkeanubeingawesome/", "https://www.reddit.com/r/DeepFakesSFW/comments/lqp4nq/my_first_ever_deepfake_billie_eilish_on_anne/", "https://old.reddit.com/r/movies/comments/osi6v3/lucasfilm_hires_the_youtube_deepfaker_who_put_its/"]
comments = []

for x in url_list:
  submission = reddit.submission(url = x)
  submission.comments.replace_more(limit=0)
  for comment in submission.comments.list():
    comment_details = {
      'title':submission.title,
      'body':comment.body,
      'votes':comment.score,
      'subreddit name':submission.subreddit.display_name,
      'time':comment.created_utc,
      'author': comment.author,
      }
    comments.append(comment_details)

for comment in comments:
  print(comment)

In [ ]:
import csv

keys = comments[0].keys()
with open('comments.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(comments)

## Getting deepfake related posts using bs4


In [ ]:
import urllib.request
from bs4 import BeautifulSoup

url = "https://old.reddit.com/search?q=url%3Adeepfake&restrict_sr=&sort=relevance&t=year"
#download the URL and extract the content to the variable html 
request = urllib.request.Request(url)
html = urllib.request.urlopen(request).read()

In [ ]:
#pass the HTML to Beautifulsoup.
soup = BeautifulSoup(html,'html.parser')
#get the HTML of the table called contents where all the links are displayed
main_table = soup.find("div",attrs={'class': 'search-result-group'})
#Now we go into main_table and get every a element in it which has a class "title" 
links = main_table.find_all("a",class_="search-title may-blank")
print(links)
metas = main_table.find_all("div",class_="search-result-meta")
print(metas[0])
scores = main_table.find_all("span",class_="search-score")
print(scores[0])
comments = main_table.find_all("a",class_="search-comments may-blank")
print(comments[0])
# dates = main_table.find_all("span",class_="search-time")
dates = main_table.find_all("time")
print(dates[0])
subreddits = main_table.find_all("a",class_="search-subreddit-link may-blank")
print(subreddits[0])

print(len(links), len(metas), len(scores), len(comments), len(dates), len(subreddits))

[<a class="search-title may-blank" href="https://old.reddit.com/r/television/comments/osa292/lucasfilm_hires_deepfake_youtuber_who_fixed/">Lucasfilm Hires Deepfake YouTuber Who Fixed Mandalorian Skywalker VFX</a>, <a class="search-title may-blank" href="https://old.reddit.com/r/technology/comments/ostv7s/lucasfilm_hires_deepfake_youtuber_who_fixed_the/">Lucasfilm hires deepfake YouTuber who fixed The Mandalorian | The YouTuber's Luke Skywalker deepfake was so good he earned himself a job.</a>, <a class="search-title may-blank" href="https://old.reddit.com/r/movies/comments/osi6v3/lucasfilm_hires_the_youtube_deepfaker_who_put_its/">Lucasfilm hires the YouTube deepfaker who put its Luke, Leia and Tarkin cameos to shame</a>, <a class="search-title may-blank" href="https://old.reddit.com/r/technews/comments/p476t5/warner_bros_reminiscence_promo_uses_deepfake_tech/">Warner Bros. 'Reminiscence' promo uses deepfake tech to put you in the trailer</a>, <a class="search-title may-blank" href="ht

In [ ]:
#from each link extract the text of link and the link itself
#List to store a dict of the data we extracted 
n = len(links)
extracted_records = []
for i in range(n): 
  title = links[i].text
  url = links[i]['href']
  score = scores[i].text
  comment = comments[i].text
  subreddit_name = subreddits[i].text
  time = dates[i]['title']
  record = {
  'title':title,
  'url':url,
  'votes':score,
  'comments':comment,
  'subreddit name':subreddit_name,
  'time':time
  }
  extracted_records.append(record)

for post in extracted_records:
  print(post)

Scripts for getting data by search query

In [ ]:
import urllib.request
from bs4 import BeautifulSoup


def find_next_url(url):
  request = urllib.request.Request(url)
  html = urllib.request.urlopen(request).read()
  soup = BeautifulSoup(html,'html.parser')
  nav_buttons = soup.find("div",attrs={'class': 'nav-buttons'})
  next = nav_buttons.find_all("a",rel="nofollow next")
  next_url = next[0]['href']
  return next_url

url = 'https://old.reddit.com/search?q=deepfake&restrict_sr=&sort=new&t=all&count=22&after=t3_p3d1g2'
for x in range(5):
  url = find_next_url(url)
  print(url)

In [ ]:

def get_reddit_data(url):
  request = urllib.request.Request(url)
  html = urllib.request.urlopen(request).read()
  soup = BeautifulSoup(html,'html.parser')
  main_table = soup.find("div",attrs={'class': 'contents'})

  links = main_table.find_all("a",class_="search-title may-blank")
  metas = main_table.find_all("div",class_="search-result-meta")
  scores = main_table.find_all("span",class_="search-score")
  comments = main_table.find_all("a",class_="search-comments may-blank")
  dates = main_table.find_all("time")
  subreddits = main_table.find_all("a",class_="search-subreddit-link may-blank")
  n = len(links)
  extracted_records = []
  for i in range(n): 
    score = '0'
    comment = '0'
    subreddit_name = 'null' 
    title = links[i].text
    url = links[i]['href']
    score = scores[i].text
    comment = comments[i].text
    subreddit_name = subreddits[i].text
    time = dates[i]['title']
    record = {
    'title':title,
    'url':url,
    'votes':score,
    'comments':comment,
    'subreddit name':subreddit_name,
    'time':time
    }
    extracted_records.append(record)
  return extracted_records


In [ ]:
# import time
# import csv

# url = "https://old.reddit.com/search?q=url%3Adeepfake&restrict_sr=&sort=relevance&t=year"

# data = []
# counter = 0
# while(True):
#   try:
#     data += get_reddit_data(url)
#     url = find_next_url(url)
#     counter += 1
#     print(counter)
#     print(len(data))
#     print(url)
#     time.sleep(10)
#   except Exception as e: 
#     print(e)
#     if e == 'list index out of range':
#       url = find_next_url(url)
#     time.sleep(60)
#     continue
#   keys = extracted_records[0].keys()
#   with open('posts.csv', 'w', newline='')  as output_file:
#       dict_writer = csv.DictWriter(output_file, keys)
#       dict_writer.writeheader()
#       dict_writer.writerows(data)

In [ ]:
keys = extracted_records[0].keys()
with open('post_data.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)

In [ ]:
print(len(data))
print(find_next_url(url))
print(url)

# Using PSAW(Pushshift)


In [ ]:
!pip install psaw
!pip install pandas

In [ ]:
from psaw import PushshiftAPI

api = PushshiftAPI()

In [ ]:
gen = api.search_submissions(limit=100)
results = list(gen)

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [ ]:
gen = api.search_submissions(q='deepfake')

max_response_cache = 100
cache = []

for c in gen:
    cache.append(c)

    # Omit this test to actually return all results. Wouldn't recommend it though: could take a while, but you do you.
    if len(cache) >= max_response_cache:
        break

# If you really want to: pick up where we left off to get the rest of the results.
if False:
    for c in gen:
        cache.append(c)

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [ ]:
print(cache)
print(len(cache))

[submission(all_awardings=[], allow_live_comments=False, author='Teos_roe', author_flair_css_class=None, author_flair_richtext=[], author_flair_text=None, author_flair_type='text', author_fullname='t2_a0g1ix2q', author_is_blocked=False, author_patreon_flair=False, author_premium=False, awarders=[], can_mod_post=False, contest_mode=False, created_utc=1629615288, crosspost_parent='t3_osj1y4', crosspost_parent_list=[{'all_awardings': [], 'allow_live_comments': False, 'approved_at_utc': None, 'approved_by': None, 'archived': False, 'author': 'Teos_roe', 'author_flair_background_color': None, 'author_flair_css_class': None, 'author_flair_richtext': [], 'author_flair_template_id': None, 'author_flair_text': None, 'author_flair_text_color': None, 'author_flair_type': 'text', 'author_fullname': 't2_a0g1ix2q', 'author_is_blocked': False, 'author_patreon_flair': False, 'author_premium': False, 'awarders': [], 'banned_at_utc': None, 'banned_by': None, 'can_gild': True, 'can_mod_post': False, 'cat

In [ ]:
import pandas as pd
gen = api.search_submissions(q='deepfake')
thing = next(gen)

df = pd.DataFrame([thing.d_ for thing in gen])
df.to_csv('file1.csv')

In [ ]:
# importing the csv library
import csv
 
# opening the csv file by specifying
# the location
# with the variable name as csv_file
with open('file1.csv') as csv_file:
 
    # creating an object of csv reader
    # with the delimiter as ,
    csv_reader = csv.reader(csv_file, delimiter = ',')
 
    # list to store the names of columns
    list_of_column_names = []
 
    # loop to iterate through the rows of csv
    for row in csv_reader:
 
        # adding the first row
        list_of_column_names.append(row)
 
        # breaking the loop after the
        # first iteration itself
        break
 
# printing the result
print("List of column names : ",
      list_of_column_names[0])

List of column names :  ['', 'all_awardings', 'allow_live_comments', 'author', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_text', 'author_flair_type', 'author_fullname', 'author_is_blocked', 'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post', 'contest_mode', 'created_utc', 'domain', 'full_link', 'gallery_data', 'gildings', 'id', 'is_created_from_ads_ui', 'is_crosspostable', 'is_gallery', 'is_meta', 'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video', 'link_flair_background_color', 'link_flair_richtext', 'link_flair_template_id', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'media_metadata', 'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18', 'permalink', 'pinned', 'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'thumbnail_height', 'thumbnail_width',

In [ ]:
# importing the module
import csv
 
# open the file in read mode
filename = open('file1.csv', 'r')
 
# creating dictreader object
file = csv.DictReader(filename)
 
# creating empty lists
links = []
titles = []
scores = []
subreddits = []
urls = []
created = []
# totalprofit = []
# totalunit = []
 
# iterating over each row and append
# values to empty list
for col in file:
    links.append(col['full_link'])
    scores.append(col['score'])
    subreddits.append(col['subreddit'])
    titles.append(col['title'])
    urls.append(col['url'])
    created.append(col['created'])



 
# printing lists
# print('links:', links)
print(len(links))
# print(len(scores), scores)
# print(len(subreddits), subreddits)
# print(len(titles), titles)
# print(len(urls), urls)
# print(len(created), created)

links: ['https://www.reddit.com/r/Cuckik/comments/p7wcf8/i_will_expose_my_sexy_and_thick_fiancé_in/', 'https://www.reddit.com/r/NSFW_Tributes/comments/p7v993/i_will_fully_expose_my_sexy_fiancé_in_exchange/', 'https://www.reddit.com/r/PewdiepieSubmissions/comments/p7v38m/someone_please_deepfake_this_with_felixs_face/', 'https://www.reddit.com/r/Cuckik/comments/p7v30v/i_will_expose_my_sexbomb_fiancé_in_exchange_for/', 'https://www.reddit.com/r/PrequelMemes/comments/p7u77v/my_first_attempt_at_a_deepfake/', 'https://www.reddit.com/r/Superstonk/comments/p7rqvo/simulation_confirmed_rc_tweet_which_has_a/', 'https://www.reddit.com/r/JadePicon/comments/p7omaq/deepfake_da_piranha/', 'https://www.reddit.com/r/LesionsOnSkanks/comments/p7ocgg/does_anyone_have_that_deepfake_pic_of_alex_and/', 'https://www.reddit.com/r/NSFWDeepfaking/comments/p7ngnm/deepfake_suggestion/', 'https://www.reddit.com/r/Cuckik/comments/p7mh4d/i_will_fully_expose_my_girl_to_anyone_who_can/', 'https://www.reddit.com/r/nsfwre

In [ ]:
import csv

counter = 0
url_list = links[18101:]
comments = []

for x in url_list:
  try:
    submission = reddit.submission(url = x)
    submission.comments.replace_more(limit=0)
    counter += 1
    print(counter)
    for comment in submission.comments.list():
      comment_details = {
        'title':submission.title,
        'body':comment.body,
        'votes':comment.score,
        'subreddit name':submission.subreddit.display_name,
        'time':comment.created_utc,
        'author': comment.author,
        }
      comments.append(comment_details)
      keys = comments[0].keys()
      with open('comments.csv', 'w', newline='')  as output_file:
          dict_writer = csv.DictWriter(output_file, keys)
          dict_writer.writeheader()
          dict_writer.writerows(comments)
      
  except Exception as e: 
    print(e)

print("DONE!")
# for comment in comments:
#   print(comment)

In [ ]:
import csv

keys = comments[0].keys()
with open('comments.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(comments)